# Traffic Sign Segmentation

**Setup**

In [1]:
#setup
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

## Function

### 1) Image Preprocessing

- **Resize image**

- **Gaussian Filtering**  
 1) To remove noise

In [2]:
def img_preprocessing (image, image_size):
    # Variable
    size = (image_size, image_size)
    kernel_size = 7
    
    # Resize image
    img = cv.resize(image, size, interpolation = cv.INTER_CUBIC)
    img_copy = img.copy()
    
    # Gaussian filter
    img_gaussian = cv.GaussianBlur(img, (kernel_size,kernel_size), 1)
    
    return img_gaussian, img_copy

### 2) Largest Contour Detection

In [3]:
def largest_contour_detect(images):
    # Find contours
    _, image = cv.threshold(images, 200, 255, cv.THRESH_TOZERO)
    contours, hierarchy = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    # Contour List
    cnt_list = np.zeros(len(contours))
    for i in range(0,len(contours)):
        cnt_list[i] = cv.contourArea(contours[i])
        
    # Find largest contour
    largest_contour_index = np.argmax(cnt_list)
    largest_contour = contours[largest_contour_index]
    contour_mask = np.zeros(image.shape, dtype = np.uint8)
    
    # Draw Contour
    if len(contours) != 0:
        cv.drawContours(contour_mask, contours, largest_contour_index, (255,255,255), -1)
        
    return image, largest_contour, contour_mask

### 3) Bounding Box Detection

In [4]:
def bounding_box(image, largest_contour):
    bounding_box = image.copy()
    x,y,w,h = cv.boundingRect(largest_contour)
    cv.rectangle(bounding_box,(x,y),(x+w,y+h),(0,255,0),2)
    
    return bounding_box

### 4) Image Segmentation
- **HSV Color segmentation**: To segment out blue color

- **Canny Edge Detection**: To detect the edge of traffic sign in mask

- **Morphological Prpcessing**
    - **Dilation** : To expand inside white region
    - **Erosion** : To decrease outside white region

In [5]:
def img_segmentation(image):
    # Go to the Bllue Traffic Sign Image folder
    path = os.getcwd() + '\\Blue Traffic Signs\\'
    
    # set Blue range and variable
    blue_low, blue_high = (90, 60, 10), (135, 255, 255)
    kernel = np.ones((3,3), np.uint8)
    
    ## Input image
    # load image
    image_ori = cv.imread(path + image)

    ## Image Preprocessing
    img, img_copy = img_preprocessing(image_ori, 300)
    
    # HSV color segmentation
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    
    # Blue mask
    mask = cv.inRange(img_hsv, blue_low, blue_high)
    
    # Canny edge detection
    img_canny = cv.Canny(mask, 50, 100)
    
    # Morphological Prpcessing
    img_Dil = cv.dilate(img_canny, kernel, iterations = 1)
    img_Ero = cv.erode(img_Dil, kernel, iterations = 1)
    
    # Find largest contour detection
    image, largest_contour, contour_mask = largest_contour_detect(img_Ero)
    
    # Bounding box detection
    img_bounding_box = bounding_box(img, largest_contour)
    
    ## Output image
    # Result
    hsv_mask = np.stack((mask,)*3, axis=-1)
    img_canny = np.stack((img_canny,)*3, axis=-1)
    seg_img = cv.bitwise_and(img_copy, img_copy, mask = contour_mask)
    final_mask = np.stack((contour_mask,)*3, axis=-1)
    combine = np.concatenate((img, hsv_mask, img_canny, final_mask, img_bounding_box, seg_img), axis = 1)
    combine = cv.resize(combine, (1050,150), interpolation = cv.INTER_CUBIC)

    #display image
    cv.imshow('Result', combine)

    cv.waitKey(0)
    cv.destroyAllWindows()

## Segmentation Result

In [6]:
img_segmentation('020_1_0005.png')